# Introduction/Business Problem
---

`Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem`


If someone is looking to open a restaurant in blumenau, which neighbourhood would you recommend?

This is the defining problem for this capstone final project. The audience would be anyone who wants to or is thinking about starting a restuarent in Blumenau. Blumenau, is a small, yet, rapidly growing city in the south of Brazil. Because the city is growing, Blumenau has become a hot spot or perfect place to begin a restaurant. 

Numerous events occur in the city that promote the ever increaseing influx of foreigners, domestic and international, such as Oktober fest. As such, Blumenau does indeed offer a prefect place to begin a restaurant.  


# Data
`Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.`

I will be using a simple table of neighbourhoods also known as bairros in portguese. The data can be acquired from the local government [website](https://www.blumenau.sc.gov.br/secretarias/secretaria-de-desenvolvimento-urbano/pagina/historia-sobre-municipio/divisa-administrativa-bairros). Foursquare can then be used after the geocoordinates of each barrio is found.

The following is the data from the table that will be scrapped using BeautifulSoup:

Sobre o Município - Bairros - Divisão Administrativa
Bairros - Divisão Administrativa
Bairro Água Verde
Bairro Badenfurt
Bairro Boa Vista
Bairro Bom Retiro
Bairro Centro
Bairro Da Glória
Bairro Do Salto
Bairro Escola Agrícola
Bairro Fidélis
Bairro Fortaleza
Bairro Fortaleza Alta
Bairro Garcia
Bairro Itoupava Central
Bairro Itoupava Norte
Bairro Itoupava Seca
Bairro Itoupavazinha
Bairro Jardim Blumenau
Bairro Nova Esperança
Bairro Passo Manso
Bairro Ponta Aguda
Bairro Progresso
Bairro Ribeirão Fresco
Bairro Salto do Norte
Bairro Salto Weissbach
Bairro Testo Salto
Bairro Tribess
Bairro Valparaíso
Bairro Velha
Bairro Velha Central
Bairro Velha Grande
Bairro Victor Konder
Bairro Vila Formosa
Bairro Vila Itoupava
Bairro Vila Nova
Bairro Vorstardt



In [1]:
# import libraries
import os
import time
import json, requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv()

client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")
version = '20180604'
limit = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore'

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import silhouette_score

import folium 

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="capstone project app")

# get html from url using lxml parser and return only data within the wikitable sortable class
resp = requests.get('https://www.blumenau.sc.gov.br/secretarias/secretaria-de-desenvolvimento-urbano/pagina/historia-sobre-municipio/divisa-administrativa-bairros').text
soup = BeautifulSoup(resp, 'lxml')
data = soup.find('div',{'id':'ultimas'})

# print('Your credentails:')
# print('CLIENT_ID: ' + client_id)
# print('CLIENT_SECRET: ' + client_secret)

address = 'Blumenau, Brazil'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Blumenau are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Blumenau are -26.9195567, -49.0658025.


In [2]:
# iterate through rows and save as multideminsional array
bairros = []
for row in data.findAll('li')[1:]:
    cells = row.find_all(['span'])
   
    try:
        
        if(cells[0].text== 'Bairro Vorstardt'):
            bairro = 'Bairro Vorstadt'
        else:
            bairro = cells[0].text
        
        
    except IndexError:
        continue

    bairros.append(bairro.rstrip())

print(bairros)

['Bairro Água Verde', 'Bairro Badenfurt', 'Bairro Boa Vista', 'Bairro Bom Retiro', 'Bairro Centro', 'Bairro Da Glória', 'Bairro Do Salto', 'Bairro Escola Agrícola', 'Bairro Fidélis', 'Bairro Fortaleza', 'Bairro Fortaleza Alta', 'Bairro Garcia', 'Bairro Itoupava Central', 'Bairro Itoupava Norte', 'Bairro Itoupava Seca', 'Bairro Itoupavazinha', 'Bairro Jardim Blumenau', 'Bairro Nova Esperança', 'Bairro Passo Manso', 'Bairro Ponta Aguda', 'Bairro Progresso', 'Bairro Ribeirão Fresco', 'Bairro Salto do Norte', 'Bairro Salto Weissbach', 'Bairro Testo Salto', 'Bairro Tribess', 'Bairro Valparaíso', 'Bairro Velha', 'Bairro Velha Central', 'Bairro Velha Grande', 'Bairro Victor Konder', 'Bairro Vila Formosa', 'Bairro Vila Itoupava', 'Bairro Vila Nova', 'Bairro Vorstadt']


In [3]:
# create dataFrame from data and assign column names
df = pd.DataFrame(bairros, columns=['Bairros'])
df['Bairros'] = df['Bairros'].map(lambda x: str(x)[7:])

df

,Bairros
0,Água Verde
1,Badenfurt
2,Boa Vista
3,Bom Retiro
4,Centro
5,Da Glória
6,Do Salto
7,Escola Agrícola
8,Fidélis
9,Fortaleza


In [4]:
# def locate(x):
#     try:
#         location = geolocator.geocode('blumenau {}'.format(x))
#         print(x, location.latitude, location.longitude)
#     except:
#         time.sleep(2)
#         location = geolocator.geocode('blumenau {}'.format(x))
#         print(x, location.latitude, location.longitude)
#     time.sleep(2)
#     return location.latitude, location.longitude

# df["Latitude"], df["Longitude"] = zip(*df["Bairros"].map(locate))

# in fact we could just use the coords file, but in case we add addition columns in future df I will leave this way
latlong = pd.read_csv('coords.csv')
df = pd.merge(df, latlong, on='Bairros')

df.head()

,Bairros,Latitude,Longitude
0,Água Verde,-26.910743,-49.107369
1,Badenfurt,-26.883060,-49.135753
2,Boa Vista,-26.901357,-49.066842
3,Bom Retiro,-26.925561,-49.071635
4,Centro,-26.919902,-49.065934


In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        params = dict(
          client_id = client_id,
          client_secret = client_secret,
          v=version,
          ll='{},{}'.format(lat,lng),
          radius=radius,
          query='Restaurant',
          limit=limit
        )

        resp = requests.get(url=url, params=params)
        data = json.loads(resp.text)

        results = data["response"]['groups'][0]['items']      
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Bairros', 
                  'Bairros Latitude', 
                  'Bairros Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

blumenau_venues = getNearbyVenues(
    names=df['Bairros'],
    latitudes=df['Latitude'],
    longitudes=df['Longitude'])

Água Verde
Badenfurt
Boa Vista
Bom Retiro
Centro
Da Glória
Do Salto
Escola Agrícola
Fidélis
Fortaleza
Fortaleza Alta
Garcia
Itoupava Central
Itoupava Norte
Itoupava Seca
Itoupavazinha
Jardim Blumenau
Nova Esperança
Passo Manso
Ponta Aguda
Progresso
Ribeirão Fresco
Salto do Norte
Salto Weissbach
Testo Salto
Tribess
Valparaíso
Velha
Velha Central
Velha Grande
Victor Konder
Vila Formosa
Vila Itoupava
Vila Nova
Vorstadt


In [6]:
print('There are {} uniques categories.'.format(len(blumenau_venues['Venue Category'].unique())))

There are 43 uniques categories.


In [7]:
# one hot encoding
blumenau_onehot = pd.get_dummies(blumenau_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
blumenau_onehot['Bairros'] = blumenau_venues['Bairros'] 

# move neighborhood column to the first column
fixed_columns = [blumenau_onehot.columns[-1]] + list(blumenau_onehot.columns[:-1])
blumenau_onehot = blumenau_onehot[fixed_columns]

blumenau_onehot.head()

,Bairros,American Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Churrascaria,Creperie,Deli / Bodega,Diner,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Hawaiian Restaurant,Hot Dog Joint,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Pastelaria,Pizza Place,Restaurant,Salad Place,Sandwich Place,Snack Place,Southern Brazilian Restaurant,Steakhouse,Sushi Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Água Verde,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Água Verde,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Badenfurt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,Badenfurt,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Badenfurt,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
blumenau_grouped = blumenau_onehot.groupby('Bairros').mean().reset_index()
blumenau_grouped

,Bairros,American Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Churrascaria,Creperie,Deli / Bodega,Diner,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Hawaiian Restaurant,Hot Dog Joint,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Pastelaria,Pizza Place,Restaurant,Salad Place,Sandwich Place,Snack Place,Southern Brazilian Restaurant,Steakhouse,Sushi Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Badenfurt,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Boa Vista,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000
2,Bom Retiro,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Centro,0.000000,0.000000,0.000000,0.016949,0.000000,0.152542,0.016949,0.084746,0.000000,0.186441,0.016949,0.000000,0.0,0.000000,0.016949,0.050847,0.000000,0.0,0.033898,0.000000,0.000000,0.00,0.016949,0.0,0.016949,0.016949,0.000000,0.067797,0.033898,0.000000,0.000000,0.000000,0.016949,0.067797,0.084746,0.000000,0.016949,0.016949,0.016949,0.000000,0.000000,0.000000,0.050847
4,Da Glória,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Do Salto,0.000000,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Escola Agrícola,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Fortaleza,0.000000,0.000000,0.000000,0.166667,0.000000,0.083333,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.166667,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.166667,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000
8,Fortaleza Alta,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [9]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [10]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Bairros']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
bairros_venues_sorted = pd.DataFrame(columns=columns)
bairros_venues_sorted['Bairros'] = blumenau_grouped['Bairros']

for ind in np.arange(blumenau_grouped.shape[0]):
    bairros_venues_sorted.iloc[ind, 1:] = return_most_common_venues(blumenau_grouped.iloc[ind, :], num_top_venues)

bairros_venues_sorted

,Bairros,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Badenfurt,Brazilian Restaurant,Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Churrascaria,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner
1,Boa Vista,Steakhouse,Pastelaria,Food Stand,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega
2,Bom Retiro,Italian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,Creperie
3,Centro,Café,Brazilian Restaurant,Restaurant,Burger Joint,Italian Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Japanese Restaurant,Food Court
4,Da Glória,BBQ Joint,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,Creperie
5,Do Salto,Bakery,Bistro,Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Churrascaria,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant
6,Escola Agrícola,Bakery,Café,Food,Creperie,Chinese Restaurant,Food Court,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega
7,Fortaleza,Bakery,Restaurant,Pizza Place,Hot Dog Joint,Italian Restaurant,Snack Place,Brazilian Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Fish & Chips Shop
8,Fortaleza Alta,Bakery,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,Creperie
9,Garcia,Bakery,Sushi Restaurant,Fast Food Restaurant,Restaurant,Deli / Bodega,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food,Fish & Chips Shop,Diner


In [23]:
RAND_STATE=50
folds=3
k_fold = KFold(n_splits=folds, shuffle=True, random_state=RAND_STATE)
hyperparams = {
    "n_clusters": [2, 3, 4],
    "n_init": [10, 15, 20],
    "max_iter": [100, 200, 300, 400, 500],
    "tol": [.0000001, .000001, .00001, .0001],
}

k_means = KMeans()

ensemble = GridSearchCV(
    estimator=k_means,
    param_grid=hyperparams,
    cv=k_fold,
    n_jobs=-1
)

blumenau_grouped_clustering = blumenau_grouped.drop('Bairros', 1)
ensemble.fit(blumenau_grouped_clustering)

labels = ensemble.predict(blumenau_grouped_clustering)
score = silhouette_score(blumenau_grouped_clustering, labels)

print(score)
print(ensemble.best_params_)

0.3336180656533405
{'max_iter': 200, 'n_clusters': 4, 'n_init': 15, 'tol': 1e-05}


In [24]:
# run k-means clustering
kmeans = KMeans(n_clusters=ensemble.best_params_['n_clusters'], max_iter=ensemble.best_params_['max_iter'], n_init=ensemble.best_params_['n_init'], tol=ensemble.best_params_['tol'] ,random_state=RAND_STATE).fit(blumenau_grouped_clustering)

print(len(kmeans.labels_), len(blumenau_grouped_clustering), len(df), len(bairros_venues_sorted))


31 31 35 31


In [32]:
blumenau_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
blumenau_merged = blumenau_merged.join(bairros_venues_sorted.set_index('Bairros'), on='Bairros')

# drop all rows with NaN
blumenau_merged = blumenau_merged.dropna()

# add clustering labels
blumenau_merged['Cluster Labels'] = kmeans.labels_

blumenau_merged.head()

,Bairros,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Água Verde,-26.910743,-49.107369,Fast Food Restaurant,Hot Dog Joint,Vegetarian / Vegan Restaurant,Food Truck,Food Court,Food,Fish & Chips Shop,Diner,Deli / Bodega,Creperie,0
1,Badenfurt,-26.883060,-49.135753,Brazilian Restaurant,Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Churrascaria,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,0
2,Boa Vista,-26.901357,-49.066842,Steakhouse,Pastelaria,Food Stand,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,0
3,Bom Retiro,-26.925561,-49.071635,Italian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,Creperie,0
4,Centro,-26.919902,-49.065934,Café,Brazilian Restaurant,Restaurant,Burger Joint,Italian Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Japanese Restaurant,Food Court,3


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(ensemble.best_params_['n_clusters'])
ys = [i+x+(i*x)**2 for i in range(ensemble.best_params_['n_clusters'])]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blumenau_merged['Latitude'], blumenau_merged['Longitude'], blumenau_merged['Bairros'], blumenau_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
blumenau_merged.loc[blumenau_merged['Cluster Labels'] == 0, blumenau_merged.columns[list(range(blumenau_merged.shape[1]))]]

,Bairros,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Água Verde,-26.910743,-49.107369,Fast Food Restaurant,Hot Dog Joint,Vegetarian / Vegan Restaurant,Food Truck,Food Court,Food,Fish & Chips Shop,Diner,Deli / Bodega,Creperie,0
1,Badenfurt,-26.883060,-49.135753,Brazilian Restaurant,Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Churrascaria,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,0
2,Boa Vista,-26.901357,-49.066842,Steakhouse,Pastelaria,Food Stand,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,0
3,Bom Retiro,-26.925561,-49.071635,Italian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,Creperie,0
5,Da Glória,-26.964187,-49.059479,BBQ Joint,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,Creperie,0
6,Do Salto,-26.883472,-49.102599,Bakery,Bistro,Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Churrascaria,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,0
7,Escola Agrícola,-26.895078,-49.099026,Bakery,Café,Food,Creperie,Chinese Restaurant,Food Court,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,0
10,Fortaleza Alta,-26.847192,-49.050457,Bakery,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,Creperie,0
11,Garcia,-26.934577,-49.059467,Bakery,Sushi Restaurant,Fast Food Restaurant,Restaurant,Deli / Bodega,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food,Fish & Chips Shop,Diner,0
12,Itoupava Central,-26.816190,-49.089223,Bakery,Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,0


In [47]:
blumenau_merged.loc[blumenau_merged['Cluster Labels'] == 1, blumenau_merged.columns[list(range(blumenau_merged.shape[1]))]]

,Bairros,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
9,Fortaleza,-26.879053,-49.065259,Bakery,Restaurant,Pizza Place,Hot Dog Joint,Italian Restaurant,Snack Place,Brazilian Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Fish & Chips Shop,1
23,Salto Weissbach,-26.896694,-49.129936,German Restaurant,Diner,Vegetarian / Vegan Restaurant,Food Truck,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Deli / Bodega,Creperie,1


In [48]:
blumenau_merged.loc[blumenau_merged['Cluster Labels'] == 2, blumenau_merged.columns[list(range(blumenau_merged.shape[1]))]]

,Bairros,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
20,Progresso,-26.972253,-49.075171,Bakery,Restaurant,Burger Joint,Deli / Bodega,Vegetarian / Vegan Restaurant,Churrascaria,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,2
24,Testo Salto,-26.849167,-49.146605,Bakery,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,Creperie,2
25,Tribess,-26.871691,-49.049748,Fried Chicken Joint,Bakery,Snack Place,Hot Dog Joint,Chinese Restaurant,Food,Fish & Chips Shop,Fast Food Restaurant,Diner,Deli / Bodega,2
27,Velha,-26.918493,-49.092507,Restaurant,Bakery,Sushi Restaurant,Italian Restaurant,Fast Food Restaurant,Brazilian Restaurant,Pizza Place,Deli / Bodega,Café,Vegetarian / Vegan Restaurant,2
31,Vila Formosa,-26.938025,-49.071479,Burger Joint,BBQ Joint,Bakery,Brazilian Restaurant,Vegetarian / Vegan Restaurant,Creperie,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant,2


In [49]:
blumenau_merged.loc[blumenau_merged['Cluster Labels'] == 3, blumenau_merged.columns[list(range(blumenau_merged.shape[1]))]]

,Bairros,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
4,Centro,-26.919902,-49.065934,Café,Brazilian Restaurant,Restaurant,Burger Joint,Italian Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Japanese Restaurant,Food Court,3


In [30]:
blumenau_merged.groupby(['Cluster Labels', '1st Most Common Venue']).count()

Bairros  Latitude  Longitude  \
Cluster Labels 1st Most Common Venue                                 
0              BBQ Joint                    2         2          2   
               Bakery                       8         8          8   
               Brazilian Restaurant         3         3          3   
               Café                         3         3          3   
               Fast Food Restaurant         1         1          1   
               Italian Restaurant           1         1          1   
               Restaurant                   3         3          3   
               Snack Place                  1         1          1   
               Steakhouse                   1         1          1   
1              Bakery                       1         1          1   
               German Restaurant            1         1          1   
2              Bakery                       2         2          2   
               Burger Joint                 1         1          1   
               Fried Chicken Joint          1         1          1   
               Restaurant                   1         1          1   
3              Café                         1         1          1   

                                      2nd Most Common Venue  \
Cluster Labels 1st Most Common Venue                          
0              BBQ Joint                                  2   
               Bakery                                     8   
               Brazilian Restaurant                       3   
               Café                                       3   
               Fast Food Restaurant                       1   
               Italian Restaurant                         1   
               Restaurant                                 3   
               Snack Place                                1   
               Steakhouse                                 1   
1              Bakery                                     1   
               German Restaurant                          1   
2              Bakery                                     2   
               Burger Joint                               1   
               Fried Chicken Joint                        1   
               Restaurant                                 1   
3              Café                                       1   

                                      3rd Most Common Venue  \
Cluster Labels 1st Most Common Venue                          
0              BBQ Joint                                  2   
               Bakery                                     8   
               Brazilian Restaurant                       3   
               Café                                       3   
               Fast Food Restaurant                       1   
               Italian Restaurant                         1   
               Restaurant                                 3   
               Snack Place                                1   
               Steakhouse                                 1   
1              Bakery                                     1   
               German Restaurant                          1   
2              Bakery                                     2   
               Burger Joint                               1   
               Fried Chicken Joint                        1   
               Restaurant                                 1   
3              Café                                       1   

                                      4th Most Common Venue  \
Cluster Labels 1st Most Common Venue                          
0              BBQ Joint                                  2   
               Bakery                                     8   
               Brazilian Restaurant                       3   
               Café                                       3   
               Fast Food Restaurant                       1   
               Italian Restaurant                         1   
   